In [2]:
from pyarrow import fs
import pyarrow.dataset as ds
import pyarrow as pa
import s3fs
import pandas as pd

# Configuração do ambiente

Variáveis:
- FILENAME: Nome do arquivo que deve ser lido do raw e armazenado no parquet. NUNCA FAZER MODIFICACOES NO RAW
- config: Dicionários com credenciais.

In [44]:
FILENAME = 'base-ceb-15-04-2025-utf8'
SOURCE_PATH = 'raw/UCTIS/bases-utf8'
TARGET_PATH='UCTIS/bases/bases-utf8/base-ceb-15-04-2025'
RA = 'aguas-claras-av-araucarias'
RESPONSAVEL_NOME = 'matheus.fernandes'
RESPONSAVEL_SETOR = 'UCTIS/COGEI/GEINO'
RESPONSAVEL_EMAIL = 'matheus.fernandes@ipe.df.gov.br'
PROCESSED = 'false'

In [ ]:
minio = s3fs.S3FileSystem(key="", # change default key 
                        secret="", # change default secret
                        use_ssl=False,
                        client_kwargs={
                            'endpoint_url': 'http://minio:9000'} # change ip
                         )
minio.ls(SOURCE_PATH)

['raw/UCTIS/bases-utf8/base-caesb-15-04-2025-utf8.csv',
 'raw/UCTIS/bases-utf8/base-ceb-15-04-2025-utf8.csv',
 'raw/UCTIS/bases-utf8/base-dflegal-15-04-2025-utf8.csv',
 'raw/UCTIS/bases-utf8/base-seduh-16-04-2025-utf8.csv',
 'raw/UCTIS/bases-utf8/base-seduh-16-04-2025-utf82.csv']

In [29]:
# Le arquivo csv do minio
with minio.open(f'{SOURCE_PATH}/{FILENAME}.csv', 'r', encoding='utf-8') as f:
    response = f.read()

# Copia para um dataframe
# response
from io import BytesIO, StringIO
df = pd.read_csv(StringIO(response), sep=';')
df

/tmp/ipykernel_87/4043432324.py:8: DtypeWarning: Columns (12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(response), sep=';')


,id,num-uc,num-cli,endereco,situacao,consumo,latitude,longitude,localidade,classe,ra_num,ra,setor2010,situacao_2,bairro_pda,40local,setor2019
0,180531,243683,243267,ST N Q 01 CJ D CS 301 GAMA,LIGADA,65,"-16,00892","-48,064665",GAMA,RESIDENCIAL,2,Gama,530010805070057,pesquisado,Setor Norte,gama,530010805070057
1,180532,243684,243268,ST N Q 01 CJ D CS 303 GAMA,LIGADA,235,"-16,008815","-48,064647",GAMA,RESIDENCIAL,2,Gama,530010805070057,pesquisado,Setor Norte,gama,530010805070057
2,180533,853700,972691,ST N Q 01 CJ D LT 214 CS 02 GAMA,LIGADA,2,"-16,008238","-48,064603",GAMA,RESIDENCIAL,2,Gama,530010805070057,pesquisado,Setor Norte,gama,530010805070057
3,180534,243699,243283,ST N Q 01 CJ D CS 403 GAMA,LIGADA,132,"-16,008222","-48,064587",GAMA,RESIDENCIAL,2,Gama,530010805070057,pesquisado,Setor Norte,gama,530010805070057
4,180535,243704,243288,ST N Q 01 CJ D CS 216 GAMA,LIGADA,88,"-16,00823","-48,064575",GAMA,RESIDENCIAL,2,Gama,530010805070057,pesquisado,Setor Norte,gama,530010805070057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,34338,55424,1789097,SQN 402 BL F AP 107 PLANO PILOTO,LIGADA,331,"-15,783304","-47,874273",BRASILIA,RESIDENCIAL,1,Plano Piloto,530010805060112,pesquisado,Asa Norte,asa norte,530010805060112
1048571,34339,55423,1639493,SQN 402 BL F AP 105 PLANO PILOTO,LIGADA,0,"-15,783304","-47,874273",BRASILIA,RESIDENCIAL,1,Plano Piloto,530010805060112,pesquisado,Asa Norte,asa norte,530010805060112
1048572,34340,55432,2006532,SQN 402 BL F AP 303 PLANO PILOTO,LIGADA,173,"-15,783321","-47,87427",BRASILIA,RESIDENCIAL,1,Plano Piloto,530010805060112,pesquisado,Asa Norte,asa norte,530010805060112
1048573,34341,55436,2002045,SQN 402 BL F AP ZEL PLANO PILOTO,DESLIGADA,0,"-15,783321","-47,87427",BRASILIA,RESIDENCIAL,1,Plano Piloto,530010805060112,pesquisado,Asa Norte,asa norte,530010805060112


In [30]:
df_copy = df.copy()

# Filtragem

Realiza a filtragem por RA

In [31]:
df_copy = df.loc[(df_copy['ra'] == '?üguas Claras') & (df_copy['classe'] == 'RESIDENCIAL')]
df_copy

,id,num-uc,num-cli,endereco,situacao,consumo,latitude,longitude,localidade,classe,ra_num,ra,setor2010,situacao_2,bairro_pda,40local,setor2019
42799,223330,780323,866429,AV ARAUCARIAS LT 305 BL B SL 151 MAGGIORE SHOP...,DESLIGADA,0,"-15,836973","-48,009837",TAGUATINGA,RESIDENCIAL,20,?üguas Claras,530010805080044,pesquisado,?üguas Claras - Vertical,aguas claras,530010805080044
42804,223335,1196271,2154279,R MANACA LT 10 AP 1103 TAGUATINGA,LIGADA,339,"-15,837112","-48,009932",TAGUATINGA,RESIDENCIAL,20,?üguas Claras,530010805080044,pesquisado,?üguas Claras - Vertical,aguas claras,530010805080044
42805,223336,1120999,1561545,R MANACA LT 10 AP 603 RES VINTAGE MANACA TAGUA...,LIGADA,329,"-15,837112","-48,009932",TAGUATINGA,RESIDENCIAL,20,?üguas Claras,530010805080044,pesquisado,?üguas Claras - Vertical,aguas claras,530010805080044
42806,223337,1117099,1553736,R MANACA LT 10 AP 108 TAGUATINGA,LIGADA,283,"-15,837112","-48,009932",TAGUATINGA,RESIDENCIAL,20,?üguas Claras,530010805080044,pesquisado,?üguas Claras - Vertical,aguas claras,530010805080044
42807,223338,1147138,1607751,R MANACA LT 10 AP 604 RES VINTAGE MANACA TAGUA...,LIGADA,263,"-15,837112","-48,009932",TAGUATINGA,RESIDENCIAL,20,?üguas Claras,530010805080044,pesquisado,?üguas Claras - Vertical,aguas claras,530010805080044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665838,846341,1056310,1452222,CA V. PIRES CH 155 LT 14 GUARA I,LIGADA,155,"-15,817953","-48,002508",GUARA,RESIDENCIAL,20,?üguas Claras,530010805160220,pesquisado,Col??nia Agr?¡cola ?üguas Claras,aguas claras,530010805160220
665839,846342,1086816,1501141,CA V. PIRES CH 56 LT 15-A CS 02 GUARA I,LIGADA,117,"-15,81755","-48,002473",GUARA,RESIDENCIAL,20,?üguas Claras,530010805160220,pesquisado,Col??nia Agr?¡cola ?üguas Claras,aguas claras,530010805160220
665840,846343,1086815,1505888,CA V. PIRES CH 56 LT 15-A CS 01 GUARA I,LIGADA,327,"-15,817558","-48,002452",GUARA,RESIDENCIAL,20,?üguas Claras,530010805160220,pesquisado,Col??nia Agr?¡cola ?üguas Claras,aguas claras,530010805160220
665841,846344,1006724,1223247,CA V. PIRES CH 56 LT 16 GUARA I,LIGADA,476,"-15,817955","-48,002447",GUARA,RESIDENCIAL,20,?üguas Claras,530010805160220,pesquisado,Col??nia Agr?¡cola ?üguas Claras,aguas claras,530010805160220


In [107]:
df2 = df_copy

# Extração

Extrai apenas as colunas que tem a ver com endereço e filtra apenas as quadras fora de nucleo rural.


In [108]:
import re
pattern = re.compile(r'^(AV ARAUCARIAS)')
df2 = df2[
    ['endereco', 'latitude', 'longitude', 'localidade', 'classe', 'bairro_pda']
]
df2 = df2.loc[
    df2['endereco'].str.contains(pattern)
]
# df2['endereco'] = df2['endereco'].str.replace('H I', 'H/I').str.replace('H /I', 'H/I').str.replace('H E I', 'H/I')
df2

/tmp/ipykernel_87/1948254143.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df2['endereco'].str.contains(pattern)


,endereco,latitude,longitude,localidade,classe,bairro_pda
42799,AV ARAUCARIAS LT 305 BL B SL 151 MAGGIORE SHOP...,"-15,836973","-48,009837",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical
45627,AV ARAUCARIAS LT 350 BL C AP 301 TAGUATINGA,"-15,83389","-48,010818",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical
46009,AV ARAUCARIAS LT 305 BL B SL 234 TAGUATINGA,"-15,837045","-48,010468",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical
46010,AV ARAUCARIAS LT 305 BL B SL 144 TAGUATINGA,"-15,837138","-48,010458",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical
46011,AV ARAUCARIAS LT 305 SL 149 ED MAJORE SHOPING ...,"-15,837118","-48,010455",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical
...,...,...,...,...,...,...
230035,AV ARAUCARIAS LT 4400 BL C AP 1903 TAGUATINGA,"-15,833303","-48,045952",TAGUATINGA,RESIDENCIAL,Taguatinga Sul
230036,AV ARAUCARIAS LT 4400 BL C AP 2001 RES. ENSEAD...,"-15,833303","-48,045952",TAGUATINGA,RESIDENCIAL,Taguatinga Sul
230037,AV ARAUCARIAS LT 4400 BL B AP 501 TAGUATINGA,"-15,83367","-48,045947",TAGUATINGA,RESIDENCIAL,Taguatinga Sul
230038,AV ARAUCARIAS LT 4400 BL A AP 1107 RES. VIA EN...,"-15,83367","-48,045947",TAGUATINGA,RESIDENCIAL,Taguatinga Sul


# Tratamento dos dados
Remoção de valores nulos e quebra da coluna de endereço nos seguintes componentes:

- identificador_setor (ST, setor);
- setor;
- identificador_quadra (QD, quadra);
- quadra;
- identificador_conjunto (CJ, conjunto, BL, bloco);
- conjunto;
- identificador_lote (LT, lote, CS, casa);
- lote;
- complemento (opcional). Ex: nome do edifício;
- regiao_administrativa,


In [109]:
# Quebra da coluna de endereco nos componentes acima
df2['identificador_setor'] = df2['endereco'].str.split(' ').str[0]
df2['setor'] = df2['endereco'].str.split(' ').str[1]

df2['identificador_quadra'] = df2['endereco'].str.extract(r'.*(LT).*')
df2['quadra'] = df2['endereco'].str.extract(r'LT (\S+) \S+')

df2['identificador_conjunto'] = df2['endereco'].str.extract(r'.*(BL).*')
df2['conjunto'] = df2['endereco'].str.extract(r'BL ([A-Z]) \S+')

df2['identificador_lote'] = df2['endereco'].str.extract(r'.*([AS][PL]).*') # Match "SL" or "AP"
df2['lote'] = df2['endereco'].str.extract(r'[AS][PL] (\d+) ')

# Renomeia colunas
# df2.rename(columns={'bairro_pda': 'setor'}, inplace=True)

df2

,endereco,latitude,longitude,localidade,classe,bairro_pda,identificador_setor,setor,identificador_quadra,quadra,identificador_conjunto,conjunto,identificador_lote,lote
42799,AV ARAUCARIAS LT 305 BL B SL 151 MAGGIORE SHOP...,"-15,836973","-48,009837",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical,AV,ARAUCARIAS,LT,305,BL,B,SL,151
45627,AV ARAUCARIAS LT 350 BL C AP 301 TAGUATINGA,"-15,83389","-48,010818",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical,AV,ARAUCARIAS,LT,350,BL,C,AP,301
46009,AV ARAUCARIAS LT 305 BL B SL 234 TAGUATINGA,"-15,837045","-48,010468",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical,AV,ARAUCARIAS,LT,305,BL,B,SL,234
46010,AV ARAUCARIAS LT 305 BL B SL 144 TAGUATINGA,"-15,837138","-48,010458",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical,AV,ARAUCARIAS,LT,305,BL,B,SL,144
46011,AV ARAUCARIAS LT 305 SL 149 ED MAJORE SHOPING ...,"-15,837118","-48,010455",TAGUATINGA,RESIDENCIAL,?üguas Claras - Vertical,AV,ARAUCARIAS,LT,305,NaN,NaN,SL,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230035,AV ARAUCARIAS LT 4400 BL C AP 1903 TAGUATINGA,"-15,833303","-48,045952",TAGUATINGA,RESIDENCIAL,Taguatinga Sul,AV,ARAUCARIAS,LT,4400,BL,C,AP,1903
230036,AV ARAUCARIAS LT 4400 BL C AP 2001 RES. ENSEAD...,"-15,833303","-48,045952",TAGUATINGA,RESIDENCIAL,Taguatinga Sul,AV,ARAUCARIAS,LT,4400,BL,C,AP,2001
230037,AV ARAUCARIAS LT 4400 BL B AP 501 TAGUATINGA,"-15,83367","-48,045947",TAGUATINGA,RESIDENCIAL,Taguatinga Sul,AV,ARAUCARIAS,LT,4400,BL,B,AP,501
230038,AV ARAUCARIAS LT 4400 BL A AP 1107 RES. VIA EN...,"-15,83367","-48,045947",TAGUATINGA,RESIDENCIAL,Taguatinga Sul,AV,ARAUCARIAS,LT,4400,BL,A,AP,1107


In [110]:
# Remove colunas
del df2['endereco']

# Carregamento

Carregamento no bucket `intermediate` para análise com outras ferramentas ou mais tratamento para carregamento no DW.

In [111]:
# Converte para parquet e sobe para o minio
from pyarrow import Table, parquet as pq
minio_url = f's3://intermediate/{TARGET_PATH}/{FILENAME}-{RA}.parquet'

pq.write_table(
    Table.from_pandas(df2),
    minio_url,
    filesystem=minio,
    use_dictionary=True,
    compression='snappy',
    version='2.6',
)

In [112]:
# Adiciona tags ao arquivo
tags = {
    'User': RESPONSAVEL_NOME,
    'Department': RESPONSAVEL_SETOR,
    'Email': RESPONSAVEL_EMAIL,
    'Processed': PROCESSED,
}
parquet_partitions = minio.ls(f'intermediate/{TARGET_PATH}/{FILENAME}-{RA}.parquet')
# parquet_partitions
for p in parquet_partitions:
    minio.put_tags(p, tags, mode='o')